In [1]:
import os
from pathlib import Path

root_directory = "/cluster/home/wueestm/f3loc/"
os.chdir(root_directory)
root_directory
Path.cwd()

PosixPath('/cluster/home/wueestm/f3loc')

In [2]:
import torch
import os
import torch
try:
    from mmcv.utils import Config, DictAction
except ImportError:
    from mmengine import Config, DictAction

from metric3d.mono.model.monodepth_model import get_configured_monodepth_model

import matplotlib.pyplot as plt
import numpy as np
import time

In [3]:
# Load checkpoint
checkpoint = torch.load('metric3d/metric_depth_vit_small_800k.pth')

In [4]:
checkpoint

{'model_state_dict': OrderedDict([('depth_model.encoder.cls_token',
               tensor([[[ 5.0859e-02,  2.3980e-03,  3.7325e-03,  3.7082e-03, -4.0549e-02,
                          1.5919e-03, -2.7654e-03,  3.2307e-03, -1.6669e-02,  4.0201e-03,
                         -5.9461e-03,  4.4575e-03,  3.6598e-02, -1.8011e-02, -1.4185e-02,
                         -1.6488e-02, -6.9107e-03, -4.8147e-03,  2.6645e-02, -7.7644e-04,
                          7.1887e-03, -2.5720e-02,  1.0405e-02, -2.1206e-03,  4.5017e-03,
                         -2.0940e-04, -1.6721e-04,  1.2949e-02,  7.8341e-03, -2.4378e-03,
                          3.5519e-03,  1.8151e-03, -2.1816e-02,  4.5900e-03,  9.8811e-03,
                         -1.5651e-02,  2.4100e-04, -1.7380e-03,  1.3897e-03,  8.9900e-03,
                         -1.8463e-03,  9.0390e-03,  5.2300e-03, -1.4483e-02,  9.1505e-03,
                         -9.3924e-03, -6.3040e-03,  4.5268e-02,  8.5610e-03,  5.0961e-03,
                          4.7371

In [5]:
checkpoint["model_state_dict"].keys()

odict_keys(['depth_model.encoder.cls_token', 'depth_model.encoder.pos_embed', 'depth_model.encoder.register_tokens', 'depth_model.encoder.patch_embed.proj.weight', 'depth_model.encoder.patch_embed.proj.bias', 'depth_model.encoder.blocks.0.0.norm1.weight', 'depth_model.encoder.blocks.0.0.norm1.bias', 'depth_model.encoder.blocks.0.0.attn.qkv.weight', 'depth_model.encoder.blocks.0.0.attn.qkv.bias', 'depth_model.encoder.blocks.0.0.attn.proj.weight', 'depth_model.encoder.blocks.0.0.attn.proj.bias', 'depth_model.encoder.blocks.0.0.ls1.gamma', 'depth_model.encoder.blocks.0.0.norm2.weight', 'depth_model.encoder.blocks.0.0.norm2.bias', 'depth_model.encoder.blocks.0.0.mlp.fc1.weight', 'depth_model.encoder.blocks.0.0.mlp.fc1.bias', 'depth_model.encoder.blocks.0.0.mlp.fc2.weight', 'depth_model.encoder.blocks.0.0.mlp.fc2.bias', 'depth_model.encoder.blocks.0.0.ls2.gamma', 'depth_model.encoder.blocks.0.1.norm1.weight', 'depth_model.encoder.blocks.0.1.norm1.bias', 'depth_model.encoder.blocks.0.1.attn.

In [6]:
#metric3d_dir = os.path.abspath(os.path.dirname(__file__))
metric3d_dir = root_directory

MODEL_TYPE = {
    'ConvNeXt-Tiny': {
        'cfg_file': os.path.join(metric3d_dir, 'metric3d/mono/configs/HourglassDecoder/convtiny.0.3_150.py'),
        'ckpt_file': os.path.join(metric3d_dir, 'metric3d/convtiny_hourglass_v1.pth'),
    },
    'ConvNeXt-Large': {
        'cfg_file': os.path.join(metric3d_dir, 'metric3d/mono/configs/HourglassDecoder/convlarge.0.3_150.py'),
        'ckpt_file': os.path.join(metric3d_dir, 'metric3d/convlarge_hourglass_0.3_150_step750k_v1.1.pth'),
    },
    'ViT-Small': {
        'cfg_file': os.path.join(metric3d_dir, 'metric3d/mono/configs/HourglassDecoder/vit.raft5.small.py'),
        'ckpt_file': os.path.join(metric3d_dir, 'metric3d/metric_depth_vit_small_800k.pth'),
    },
    'ViT-Large': {
        'cfg_file': os.path.join(metric3d_dir, 'metric3d/mono/configs/HourglassDecoder/vit.raft5.large.py'),
        'ckpt_file': os.path.join(metric3d_dir, 'metric3d/metric_depth_vit_large_800k.pth'),
    },
    'ViT-giant2': {
        'cfg_file': os.path.join(metric3d_dir, 'metric3d/mono/configs/HourglassDecoder/vit.raft5.giant2.py'),
        'ckpt_file': os.path.join(metric3d_dir, 'metric3d/metric_depth_vit_giant2_800k.pth'),
    },
}

def load_model(model_type, pretrain=False, **kwargs):
    '''
    Load a Metric3D model based on the specified model type.
    Args:
        model_type (str): Type of the model to load.
        pretrain (bool): Whether to load pretrained weights.
    Returns:
        model (nn.Module): A Metric3D model.
    '''
    cfg_file = MODEL_TYPE[model_type]['cfg_file']
    ckpt_file = MODEL_TYPE[model_type]['ckpt_file']

    cfg = Config.fromfile(cfg_file)
    model = get_configured_monodepth_model(cfg)
    if pretrain:
        model.load_state_dict(
            torch.load(ckpt_file)['model_state_dict'],  # Use torch.load for local file
            strict=False,
        )
    return model

def metric3d_convnext_tiny(pretrain=False, **kwargs):
    return load_model('ConvNeXt-Tiny', pretrain, **kwargs)

def metric3d_convnext_large(pretrain=False, **kwargs):
    return load_model('ConvNeXt-Large', pretrain, **kwargs)

def metric3d_vit_small(pretrain=False, **kwargs):
    return load_model('ViT-Small', pretrain, **kwargs)

def metric3d_vit_large(pretrain=False, **kwargs):
    return load_model('ViT-Large', pretrain, **kwargs)

def metric3d_vit_giant2(pretrain=False, **kwargs):
    return load_model('ViT-giant2', pretrain, **kwargs)

In [7]:
model = metric3d_vit_small(pretrain=True)

/cluster/home/wueestm/anaconda3/envs/f3loc/lib/python3.8/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
xFormers not available
xFormers not available
xFormers not available
xFormers not available


In [8]:
model

DepthModel(
  (depth_model): DensePredModel(
    (encoder): DinoVisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
        (norm): Identity()
      )
      (blocks): ModuleList(
        (0): BlockChunk(
          (0-11): 12 x Block(
            (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
            (attn): MemEffAttention(
              (qkv): Linear(in_features=384, out_features=1152, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=384, out_features=384, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
            )
            (ls1): LayerScale()
            (drop_path1): Identity()
            (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=384, out_features=1536, bias=True)
              (act): GELU(approximate='none')
              (fc

In [9]:
import torchvision
from torchview import draw_graph

model_graph = draw_graph(resnet18(), input_size=(1,3,224,224), expand_nested=True)
model_graph.visual_graph

ModuleNotFoundError: No module named 'torchview'

In [15]:
dependencies = ['torch', 'torchvision']

import os
import torch
try:
    from mmcv.utils import Config, DictAction
except ImportError:
    from mmengine import Config, DictAction

from metric3d.mono.model.monodepth_model import get_configured_monodepth_model

import matplotlib.pyplot as plt
import numpy as np
import time


print("before first imports")
import cv2
import numpy as np
#### prepare data
#rgb_file = 'data/kitti_demo/rgb/0000000050.png'
#depth_file = 'data/kitti_demo/depth/0000000050.png'
#intrinsic = [707.0493, 707.0493, 604.0814, 180.5066]
#gt_depth_scale = 256.0
print("after first imports")
#model_name_ls = ["ViT-Small", "ViT-Large", "ViT-giant2"]
#rgb_filename_ls = ['00000-0', '00090-0', '00120-0']
model_name_ls = ["ViT-Small"]
rgb_filename_ls = ['00000-0']
#rgb_filename_ls = ['aligned_image_0', 'aligned_image_90', 'aligned_image_120']
    
model_name = model_name_ls[0]
rgb_filename = rgb_filename_ls[0]

#rgb_file = 'data/hge_customized_complete/gravity-aligned/rgb/' + rgb_filename + ".png"
rgb_file = 'metric3d/data/hge_customized_complete/non-aligned/rgb/' + rgb_filename + ".png"
depth_file = None
intrinsic = [1596, 1596, 960, 720] #[240, 240, 1440/2, 1920/2] # [240, 240, 320, 240]
rgb_origin = cv2.imread(rgb_file)[:, :, ::-1]

#### adjust input size to fit pretrained model
# keep ratio resize
input_size = (616, 1064) # for vit model
# input_size = (544, 1216) # for convnext model
h, w = rgb_origin.shape[:2]
scale = min(input_size[0] / h, input_size[1] / w)
rgb = cv2.resize(rgb_origin, (int(w * scale), int(h * scale)), interpolation=cv2.INTER_LINEAR)
# remember to scale intrinsic, hold depth
intrinsic = [intrinsic[0] * scale, intrinsic[1] * scale, intrinsic[2] * scale, intrinsic[3] * scale]
# padding to input_size
padding = [123.675, 116.28, 103.53]
h, w = rgb.shape[:2]
pad_h = input_size[0] - h
pad_w = input_size[1] - w
pad_h_half = pad_h // 2
pad_w_half = pad_w // 2
rgb = cv2.copyMakeBorder(rgb, pad_h_half, pad_h - pad_h_half, pad_w_half, pad_w - pad_w_half, cv2.BORDER_CONSTANT, value=padding)
pad_info = [pad_h_half, pad_h - pad_h_half, pad_w_half, pad_w - pad_w_half]
print("after padding of input size")
#### normalize
mean = torch.tensor([123.675, 116.28, 103.53]).float()[:, None, None]
std = torch.tensor([58.395, 57.12, 57.375]).float()[:, None, None]
rgb = torch.from_numpy(rgb.transpose((2, 0, 1))).float()
rgb = torch.div((rgb - mean), std)
rgb = rgb[None, :, :, :]#.cuda()
print("after normalize")
###################### canonical camera space ######################
# inference
if model_name == "ViT-Small":
    model = metric3d_vit_small(pretrain=True)
elif model_name == "ViT-Large":
    model = metric3d_vit_large(pretrain=True)
elif model_name == "ViT-giant2":
    model = metric3d_vit_giant2(pretrain=True)
print("after loading of model")
#model.cuda().eval()
model.eval()
start_time = time.time()
with torch.no_grad():
    pred_depth, confidence, output_dict = model.inference({'input': rgb})

before first imports
after first imports
after padding of input size
after normalize
after loading of model


RuntimeError: No CUDA GPUs are available

In [16]:
import torch
from torchviz import make_dot

# Assuming your model is already loaded and named `model`
# Create a dummy input tensor with the same shape as your model's input
dummy_input = {'input': rgb} #torch.randn(1, 3, 224, 224)  # Example for a model expecting input of shape [1, 3, 224, 224]

# Perform a forward pass to get the model's output
output = model(dummy_input)

# Create a visualization of the computational graph
dot = make_dot(output, params=dict(model.named_parameters()))

# Save the graph to a file
dot.render("model_visualization", format="png")

RuntimeError: No CUDA GPUs are available